In [3]:
import os
import shutil
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt


global_seed = 123
np.random.seed(global_seed)
tf.random.set_seed(global_seed)

In [4]:
# Download dataset

url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')



84125825/84125825 [==============================] - 8s 0us/step


In [5]:
# Remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [6]:
# Train test split

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [7]:
# View dataset

for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

Review: b'This movie is all ultra-lightweight fluff, predictable from beginning to end. As a Don Knotts vehicle, "The Incredible Mr. Limpet" was much better, with Knott\'s character there not nearly as incompetent or ignorant. His performance there was toned down, with none of his trademark goggle-eyed stare, although that may have something to do with him being replaced for most of the movie by a cartoon fish. Knotts made a living of playing the likable imbecile, much as Bob Denver did. Neither really seemed to be able to break out to other types of roles, assuming they were simply typecast. It was probably because of the slouch, the wild stare and the high-pitched voice. John Ritter, whom Knotts worked with in "Three\'s Company," was able to transcend his genre, branching out successfully into dramatic roles like "The Dreamer of Oz," but the closest Knotts ever got was a small role in "Pleasantville." Even Leslie Nielsen was a bad fit here, uncomfortably neither straight dramatic act

In [8]:
# Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'

tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1"
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [19]:
# Example of preprocessing sentence

text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

In [21]:
# Example of bert encoder output

bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
# print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
# print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Pooled Outputs Shape:(1, 128)
Sequence Outputs Shape:(1, 128, 128)


In [22]:
# Create sentiment classification function

def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [23]:
# Prediction without fine-tuned model

classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(text_test)
print(tf.sigmoid(bert_raw_result))

['this is such an amazing movie!']
tf.Tensor([[0.8020936]], shape=(1, 1), dtype=float32)


In [24]:
# Fine tuned model

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 3
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [25]:
classifier_model.compile(optimizer=optimizer,loss=loss, metrics=metrics)

In [26]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Epoch 1/3
625/625 [==============================] - 98s 142ms/step - loss: 0.5843 - binary_accuracy: 0.6599 - val_loss: 0.4584 - val_binary_accuracy: 0.7682
Epoch 2/3
625/625 [==============================] - 71s 113ms/step - loss: 0.4403 - binary_accuracy: 0.7863 - val_loss: 0.4195 - val_binary_accuracy: 0.8046
Epoch 3/3
625/625 [==============================] - 68s 109ms/step - loss: 0.3881 - binary_accuracy: 0.8177 - val_loss: 0.4004 - val_binary_accuracy: 0.8140


In [ ]:
# loss, accuracy = classifier_model.evaluate(test_ds)

# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')

In [27]:
sentences = [
    "This movie was fantastic!",
    "I didn't enjoy the film at all.",
    "The plot was quite intriguing.",
    "Characters were not well-developed.",
    "I would recommend this to my friends."
]
sentences_ds = tf.data.Dataset.from_tensor_slices(sentences).batch(batch_size)
predictions = classifier_model.predict(sentences_ds)
probabilities = tf.sigmoid(predictions)
print(probabilities.numpy())


1/1 [==============================] - 0s 320ms/step
[[0.98206884]
 [0.96360797]
 [0.65029836]
 [0.785024  ]
 [0.8431065 ]]
